# Companion Notebook
The `NSGA2Generator` can be run on multi-objective optimization problems purely from Xopt's YAML configuration files. This notebook demonstrates how to set up the configuration file, how to run the optimization algorithm, and how to load the data into python and plot the results. At the end, we demonstrate how to restart an optimization from one of `NSGA2Generator`'s checkpoint files.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import shutil

## Calling the Runner Script
We have prepared a python script and YAML configuration file in the example directory accompanying this notebook. The script simply loads the YAMl file into an Xopt object and calls `Xopt.run()` on it to perform the optimization until the termination condition is achieved.

In [ ]:
# Run the optimizer (will take ~30s)
# We will run from the example directory containing `eval_fun.py` which is imported during our xopt run
! cd assets/yaml_runner_example && python run_xopt.py nsga2_zdt3.yml 

## Loading and Plotting Data
The YAML file specified that data will be output to the directory `nsga2_output` within the working directory of the script (ie `assets/yaml_runner_example`). In here, there will be several files.
- `populations.csv`: Each completed population is recorded to this file
- `data.csv`: Contains all evaluated inviduals
- `log.txt`: A record of all log messages the genreator emitted during its run
- `vocs.txt`: A copy of the variable, objectives, and constraints (VOCs) definitions
- `checkpoints/`: This directory contains checkpoint files which are used with the `checkpoint_file` key of the generator to restart an optimization.

In [ ]:
# Load every generation
df = pd.read_csv("assets/yaml_runner_example/nsga2_output/populations.csv")
df.head()

In [ ]:
# Grab just the final generation
last_gen = df[df["xopt_generation"] == df["xopt_generation"].max()]
last_gen.head()

In [ ]:
# Plot the objectives
plt.scatter(last_gen["f1"], last_gen["f2"])
plt.xlabel("f1 (unitless)")
plt.ylabel("f2 (unitless)")
plt.title(f"Generation {last_gen['xopt_generation'].max()}")

## Restoring from Checkpoints
In this section, we will restart the optimizer from one of its saved checkpoints. This allows us to carry on an optimization that was previously terminated with no loss of information. The checkpoint is specified in the YAML file using the key `checkpoint_file`. Any additional settings in the generator will override the settings included in the checkpoint. The evaluation function still needs to be defined and should be identical to what was used during the original run.

In [ ]:
# Hack so you do not need to change the checkpoint filename in the config file manually for this tutorial
# In a real optimization, you will set `checkpoint_file` to the file of your choice
! cd assets/yaml_runner_example/nsga2_output/checkpoints &&  mv $(ls -1 | tail -n 1) 20250805_065102_1.txt

In [ ]:
# Run the optimizer for another few generations (will take ~30s)
! cd assets/yaml_runner_example && python run_xopt.py nsga2_from_checkpoint.yml 

In [ ]:
# Grab the last generation
df = pd.read_csv(
    "assets/yaml_runner_example/nsga2_from_checkpoint_output/populations.csv"
)
last_gen_restarted = df[df["xopt_generation"] == df["xopt_generation"].max()]

# Plot the objectives
plt.scatter(
    last_gen["f1"],
    last_gen["f2"],
    label=f"Generation {last_gen['xopt_generation'].max()}",
)
plt.scatter(
    last_gen_restarted["f1"],
    last_gen_restarted["f2"],
    label=f"Generation {last_gen_restarted['xopt_generation'].max()}",
)
plt.xlabel("f1 (unitless)")
plt.ylabel("f2 (unitless)")
plt.legend()

## Cleanup
We will now remove the files created in this notebook.

In [ ]:
# Clean up the output directoris
shutil.rmtree("assets/yaml_runner_example/nsga2_output")
shutil.rmtree("assets/yaml_runner_example/nsga2_from_checkpoint_output")